นาย ปพนธ์ ชุณหคล้าย 6210503691

In [ ]:
import json
from time import sleep

from bs4 import BeautifulSoup

In [ ]:

from kafka import KafkaConsumer, KafkaProducer

First it will consume data from raw_recipes topic, parse and transform data into JSON and then will publish it in parsed_recipes topic. Below is the code that will fetch HTML data from raw_recipes topic, parse and then feed into parsed_recipes topic.

In [ ]:
def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding='utf-8') 
        value_bytes = bytes(value, encoding='utf-8') 
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully (producer).')
    except Exception as ex:
        print('Exception in publishing message')
        print(str(ex))


def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'], api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return _producer


def parse(markup):
    title = '-'
    submit_by = '-'
    description = '-'
    calories = 0
    ingredients = []
    rec = {}

    try:

        soup = BeautifulSoup(markup, 'lxml')
        # title
        title_section = soup.select('.breadcrumb_last')
        # submitter
        #submitter_section = soup.select('.submitter__name')
        # description
        #description_section = soup.select('.submitter__description')
        # ingredients
        # ingredients_section = soup.select('.list--wrapper')

        # calories
        nutrition_section = soup.select('.faqs--item')#soup.select('.calorie-count')
        nutrition_section = [s  for div in nutrition_section for s in div.stripped_strings]
        print(nutrition_section)
        if nutrition_section:
            #calories = calories_section[0].text.replace('cals', '').strip()
            y=[float(nutrition_section[1])]
            for i in range (3,23,2):
              p=str(nutrition_section[i])
              if 'g' in p:
                k=p.replace('g', '')
                j=k.replace('m', '')
                y.append(float(j))
        
        # if ingredients_section:
        #     for ingredient in ingredients_section:
        #         ingredient_text = ingredient.text.strip()
        #         if 'Add all ingredients to list' not in ingredient_text and ingredient_text != '':
        #             ingredients.append({'step': ingredient.text.strip()})

        #if description_section:
        #    description = description_section[0].text.strip().replace('"', '')

        #if submitter_section:
        #    submit_by = submitter_section[0].text.strip()

        if title_section:
            title = title_section[0].text

        x = ['Calories','Protein','Total Fat','Sat Fat','Trans Fat','Cholesterol','Total Carbohydrate','Dietary fiber','Total sugars','Added sugars','Sodium']
        b = 'bar'
        rec = {'x': x,  'y': y, 'type': b, 'name': title}
        # rec = {'title': title,  'calories': calories,
        #        'ingredients': ingredients}
        print(rec)
    except Exception as ex:
        print('Exception while parsing')
        print(str(ex))
    finally:
        return json.dumps(rec)




In [ ]:
if __name__ == '__main__':
    print('Running Consumer..')
    parsed_records = []
    topic_name = 'raw_recipes'
    parsed_topic_name = 'parsed_recipes'

    consumer = KafkaConsumer(topic_name, auto_offset_reset='earliest',
                             bootstrap_servers=['localhost:9092'], api_version=(0, 10), consumer_timeout_ms=1000)
    for msg in consumer:
        html = msg.value
        result = parse(html)
        print (result)
        parsed_records.append(result)
    consumer.close()
    sleep(5)

    if len(parsed_records) > 0:
        print('Publishing records..')
        producer = connect_kafka_producer()
        for rec in parsed_records:
            publish_message(producer, parsed_topic_name, 'parsed', rec)

Running Consumer..
['Calories:', '208.1', 'Protein:', '6.4g', 'Total Fat:', '3.5g', 'Sat Fat:', '0.5g', 'Trans Fat:', '0g', 'Cholesterol:', '0mg', 'Total Carbohydrate:', '39.8g', 'Dietary fiber:', '5.4g', 'Total sugars:', '15.6g', 'Added sugars:', '1.5g', 'Sodium:', '661.6mg', 'Calories:', '208.1', 'Protein:', '6.4g', 'Total Fat:', '3.5g', 'Sat Fat:', '0.5g', 'Trans Fat:', '0g', 'Cholesterol:', '0mg', 'Total Carbohydrate:', '39.8g', 'Dietary fiber:', '5.4g', 'Total sugars:', '15.6g', 'Added sugars:', '1.5g', 'Sodium:', '661.6mg']
{'x': ['Calories', 'Protein', 'Total Fat', 'Sat Fat', 'Trans Fat', 'Cholesterol', 'Total Carbohydrate', 'Dietary fiber', 'Total sugars', 'Added sugars', 'Sodium'], 'y': [208.1, 6.4, 3.5, 0.5, 0.0, 0.0, 39.8, 5.4, 15.6, 1.5, 661.6], 'type': 'bar', 'name': 'BBQ Chickpea and Carrot Sliders'}
{"x": ["Calories", "Protein", "Total Fat", "Sat Fat", "Trans Fat", "Cholesterol", "Total Carbohydrate", "Dietary fiber", "Total sugars", "Added sugars", "Sodium"], "y": [208.

you may delete buffer topic:

bin/kafka-topics.sh --zookeeper localhost:2181 --delete --topic parsed_recipes


In [ ]:
import json
from time import sleep

from kafka import KafkaConsumer

if __name__ == '__main__':
    parsed_topic_name = 'parsed_recipes'
    # Notify if a recipe has more than 200 calories
    calories_threshold = 200

    consumer = KafkaConsumer(parsed_topic_name, auto_offset_reset='earliest',
                             bootstrap_servers=['localhost:9092'], api_version=(0, 10), consumer_timeout_ms=1000)
    for msg in consumer:
        record = json.loads(msg.value)
        print(record)
        # calories =  record['calories']
        # title = record['title']

        # if calories > calories_threshold:
        #     print('Alert: {} calories count is {}'.format(title, calories))
        # sleep(3)

    if consumer is not None:
        consumer.close()

{'x': ['Calories', 'Protein', 'Total Fat', 'Sat Fat', 'Trans Fat', 'Cholesterol', 'Total Carbohydrate', 'Dietary fiber', 'Total sugars', 'Added sugars', 'Sodium'], 'y': [208.1, 6.4, 3.5, 0.5, 0.0, 0.0, 39.8, 5.4, 15.6, 1.5, 661.6], 'type': 'bar', 'name': 'BBQ Chickpea and Carrot Sliders'}
{'x': ['Calories', 'Protein', 'Total Fat', 'Sat Fat', 'Trans Fat', 'Cholesterol', 'Total Carbohydrate', 'Dietary fiber', 'Total sugars', 'Added sugars', 'Sodium'], 'y': [262.5, 12.4, 1.4, 0.18, 0.0, 0.0, 53.4, 14.0, 9.42, 0.0, 76.6], 'type': 'bar', 'name': 'Southwest Stew'}
{'x': ['Calories', 'Protein', 'Total Fat', 'Sat Fat', 'Trans Fat', 'Cholesterol', 'Total Carbohydrate', 'Dietary fiber', 'Total sugars', 'Added sugars', 'Sodium'], 'y': [322.1, 13.2, 2.3, 0.0, 0.0, 29.9, 10.9, 8.6, 3.0, 456.7], 'type': 'bar', 'name': 'The Whole Bowl: Pan Roasted Tofu, Broccoli Bites and Wilted Spinach'}
{'x': ['Calories', 'Protein', 'Total Fat', 'Sat Fat', 'Trans Fat', 'Cholesterol', 'Total Carbohydrate', 'Dietary 